In [1]:
from piper import piper
from piper.defaults import *
from piper.verbs import *
from piper.factory import get_sample_data

piper version 0.0.7, last run: Friday, 19 February 2021 11:27:43


In [2]:
import pandas as pd
import numpy as np


def calc_weights(weights):
    ''' 
    Calculate proportion/weighting of probabilities
    '''   
    return list(map(lambda x: x / sum(weights), weights))


def get_skus(astype='list'):

    skus = ['Apple iPhone 11', 'Apple iPhone 11 Pro', 'Apple iPhone 11 Pro Max',
            'Apple iPhone XR', 'Apple iPhone XS', 'Apple iPhone XS Max',
            'Galaxy S10+', 'Galaxy S10e', 'Huawei Mate 20', 'Huawei Mate 20 Pro',
            'Huawei P20', 'Huawei P20 Pro', 'Huawei P20 lite', 'Huawei P30',
            'Huawei P30 Pro', 'Samsung Galaxy A10', 'Samsung Galaxy A20',
            'Samsung Galaxy A50', 'Samsung Galaxy J2 Core', 'Samsung Galaxy S10',
            'Samsung Galaxy S9', 'Samsung Galaxy S9+', 'Xiaomi Redmi Note 7',
            'Xiaomi Redmi Note 7 Pro', 'Xiaomi Redmi Note 8', 'Xiaomi Redmi Note 8 Pro']
    
    price = pd.Series(np.random.uniform(low=400, high=800, size=len(skus)).round(2))

    skus_data = [{'sku': sku, 'unit_price': price[idx]} for idx, sku in enumerate(skus)]
    
    if astype == 'list':
        return skus
    
    if astype == 'dataframe':
        return pd.DataFrame(skus_data)
    
    return skus_data

In [3]:
import pandas as pd
import numpy as np


def get_sample_data2(year='2020', freq='D', rows=1000, seed=None):
    ''' 
    
    '''
    if seed is not None:
        np.random.seed(seed)

    month = np.random.randint(1, 13, size=rows)
    day = np.random.randint(1, 28, size=rows)
    order_dates = pd.DataFrame({'year': year, 'month': month, 'day': day})
    
    # Calculate random day intervals
    f = lambda x: pd.Timedelta(value=x, unit='days')
    f = np.vectorize(f)
    
    order_dates = pd.to_datetime(order_dates) 
    invoice_dates = order_dates + pd.Series(f(np.random.randint(1, 3, size=rows)))
    delivery_dates = invoice_dates + pd.Series(f(np.random.randint(3, 5, size=rows))) 

    orders = pd.Series(np.random.randint(low=4600000, high=4800000, size=rows))

    country_list = ['Germany', 'Italy', 'France', 'Spain', 'Sweden', 
                    'Portugal', 'Norway', 'Switzerland']
    weights = calc_weights([3, 1, 2, 1, 1, 1, 1, 1])
    countries = pd.Series(np.random.choice(country_list, p=weights, size=rows))

    region_list = ['East', 'West', 'North', 'South']
    weights = calc_weights([4, 2, 4, 1])
    regions = pd.Series(np.random.choice(region_list, p=weights, size=rows))

    reps = ['Mr. D. Davis', 'Mr. B. Johnson', 'Mr S. Baker',
            'Mr K. Starmer', 'Mrs T. May', 'Mrs R. Johnson']
    weights = calc_weights([1, 3, 2, 2, 2, 1])
    reps = pd.Series(np.random.choice(reps, p=weights, size=rows))

    df_skus = get_skus(astype='dataframe')
    skus = pd.Series(np.random.randint(low=0, high=df_skus.shape[0], size=rows))
    skus = skus.apply(lambda x: df_skus.iloc[x, 0])
    qty = pd.Series(np.random.randint(low=1, high=40, size=rows))

    data_dictionary = {
        'country': countries,
        'region': regions,
        'order': orders,
        'order_dt': order_dates,
        'invoice_dt': invoice_dates,
        'delivery_dt': delivery_dates,
        'rep': reps,
        'sku': skus,
        'qty': qty
    }

    df = pd.DataFrame(data_dictionary)    
#     df = df.merge()

    return df

In [4]:
df = get_sample_data2(year=2022, freq='D', rows=10000)
df['duration (order to cash)'] = df.invoice_dt - df.order_dt
df['duration (order to delivery)'] = df.delivery_dt - df.order_dt
df

,country,region,order,order_dt,invoice_dt,delivery_dt,rep,sku,qty,duration (order to cash),duration (order to delivery)
0,Germany,East,4716237,2022-09-15,2022-09-17,2022-09-20,Mr. B. Johnson,Samsung Galaxy S10,19,2 days,5 days
1,France,West,4733453,2022-07-15,2022-07-17,2022-07-20,Mr. B. Johnson,Samsung Galaxy A50,6,2 days,5 days
2,France,West,4771857,2022-03-10,2022-03-11,2022-03-14,Mr. D. Davis,Samsung Galaxy S9,37,1 days,4 days
3,Switzerland,East,4736902,2022-04-11,2022-04-12,2022-04-15,Mr K. Starmer,Galaxy S10+,21,1 days,4 days
4,Switzerland,East,4679748,2022-11-24,2022-11-26,2022-11-29,Mr S. Baker,Apple iPhone 11 Pro,22,2 days,5 days
...,...,...,...,...,...,...,...,...,...,...,...
9995,Portugal,East,4622350,2022-02-18,2022-02-20,2022-02-23,Mr K. Starmer,Samsung Galaxy J2 Core,14,2 days,5 days
9996,Norway,North,4728914,2022-04-22,2022-04-23,2022-04-27,Mr. B. Johnson,Samsung Galaxy A20,16,1 days,5 days
9997,Germany,North,4736227,2022-12-19,2022-12-21,2022-12-24,Mr S. Baker,Samsung Galaxy S9,28,2 days,5 days
9998,France,East,4735326,2022-12-15,2022-12-17,2022-12-20,Mr. B. Johnson,Huawei P30 Pro,21,2 days,5 days


In [5]:
%%piper

df >> counts('rep') >> adorn()

,n
Mr. B. Johnson,2723
Mr K. Starmer,1904
Mrs T. May,1757
Mr S. Baker,1754
Mrs R. Johnson,947
Mr. D. Davis,915
All,10000
